In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mlp
import seaborn as sns
from scipy.spatial.transform import Rotation

import utils
from importlib import reload

sns.set_theme(palette=sns.color_palette("twilight_shifted"))

In [2]:
r1 = r2 = np.identity(3)
t1, t2 = np.array([0, 0, 1]).reshape(-1, 1), np.array([0, 0, 20]).reshape(-1, 1)
k1 = k2 = np.array([
    [700, 0, 600],
    [0, 700, 400],
    [0, 0, 1]
])
Q = np.array([1, 1, 0])

## **Exercise 5.1** ProjectionMatrices

In [3]:
p1 = utils.make_projection_matrix(k1, r1, t1)
p2 = utils.make_projection_matrix(k2, r2, t2)

q1 = utils.hom_to_inhom(
    utils.projectpoints(k1, r1, t1, Q)
)
q2 = utils.hom_to_inhom(
    utils.projectpoints(k2, r2, t2, Q)
)
np.set_printoptions(suppress=True)
print(f"{p1=}\n{p2=}\n{q1=}\n{q2=}")

p1=array([[700.,   0., 600., 600.],
       [  0., 700., 400., 400.],
       [  0.,   0.,   1.,   1.]])
p2=array([[  700.,     0.,   600., 12000.],
       [    0.,   700.,   400.,  8000.],
       [    0.,     0.,     1.,    20.]])
q1=array([[1.18181818]])
q2=array([[1.45977011]])


## **Exercise 5.2** Adding noise to points

In [4]:
reload(utils)
# Adding noise to points
noise = np.array([1, -1]).reshape(-1, 1)
qnoise1 = q1 + noise
qnoise2 = q2 + noise

# Stacking them for the triangulation
qs = np.vstack([qnoise1, qnoise2])
ps = np.vstack([p1, p2])

# Triangulting the point
Qnoise = utils.hom_to_inhom(
    utils.triangulate(qs, ps, return_hom=True)
)

# Reprojecting points
q1noise = utils.projectpoints(k1, r1, t1, Qnoise)
q2noise = utils.projectpoints(k2, r2, t2, Qnoise)

print(f"Triangulated point q:\n\t{Qnoise}")
print(f"Distance between noise image in p1:\n\t{np.linalg.norm(qnoise1 - q1noise):.3f} pixels")
print(f"Distance between noise image in p2:\n\t{np.linalg.norm(qnoise2 - q2noise):7.3f} pixels")
print(f"Distance original point and triangulated point:\n\t{np.linalg.norm(Q - Qnoise):.3f}")

Triangulated point q:
	[-30252.78866817 -20230.60206522  35421.47278714]
Distance between noise image in p1:
	0.038 pixels
Distance between noise image in p2:
	  0.038 pixels
Distance original point and triangulated point:
	50786.713


## **Exercise 5.3** Nonlinear triangulation

In [5]:
reload(utils)
utils.triangulate_nonlin(qs, ps)

IndexError: tuple index out of range

In [ ]:
Qtilde = utils.triangulate(qs, ps, return_hom=True)
utils.hom_to_inhom((ps @ Qtilde).reshape(-1, 3).T).T.reshape(-1)

array([1310.48950027, 1089.4923513 ,  635.53411968,  434.4839772 ])

In [ ]:
utils.hom_to_inhom((ps @ Qtilde).reshape(-1, 3).T)

array([[1310.48950027,  635.53411968],
       [1089.4923513 ,  434.4839772 ]])

In [ ]:
p1 @ Qtilde, p2 @ Qtilde

(array([756.63331027, 629.03686302,   0.57736694]),
 array([7336.73589882, 5015.77192206,   11.54420459]))